# Process the images inside the original FooDD folder

The FooDD folder contains subfolder with each class' name and corresponding folders

E.g. `FooDD/Apple` contains all pictures that have an Apple inside it

Todo:
- Rename all pictures from `Food_Name/Device_Used-Lighting_Condition/Image_number.jpg` to the same standard
    - Use `Food_NameDeviceNumber_ImageNumber.jpg` (e.g. Apple1_1.jpg - Device 1, image number 1 of the Apple)
- Scale down each image - 1000px on smallest side
- Split into training and testing (80:20)


We also need to be able to process csv files. This is to be able to read calorie and density values for each food

In [1]:
%matplotlib inline

import cv2
import csv
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets

from os import listdir
from os.path import isfile, join

In [2]:
class_label =  {0: 'Apple',
                1: 'Banana',
                2: 'Bean',
                3: 'Bread',
                4: 'Carrot',
                5: 'Cheese',
                6: 'Cucumber',
                7: 'Egg',
                8: 'Grape',
                9: 'Kiwi',
                10: 'Onion',
                11: 'Orange',
                12: 'Pasta',
                13: 'Pepper',
                14: 'Sauce',
                15: 'Tomato',
                16: 'Watermelon'}

class_label

{0: 'Apple',
 1: 'Banana',
 2: 'Bean',
 3: 'Bread',
 4: 'Carrot',
 5: 'Cheese',
 6: 'Cucumber',
 7: 'Egg',
 8: 'Grape',
 9: 'Kiwi',
 10: 'Onion',
 11: 'Orange',
 12: 'Pasta',
 13: 'Pepper',
 14: 'Sauce',
 15: 'Tomato',
 16: 'Watermelon'}

#### Read CSV files

In [3]:
def load_food_information(file):
    food_calories = {}
    food_density = {}
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:        
                food_calories[row[0]] = row[1]
                food_density[row[0]] = row[2]
                line_count += 1
    print(f'Processed {line_count} lines.')
    return food_calories,food_density

In [4]:
calories, density = load_food_information('Dataset/calories_density.csv')
print()
print('Calories: {0}\n'.format(calories))
print('Density: {0}'.format(density))

Column names are Food, Calories, Density
Processed 18 lines.

Calories: {'Apple': '52', 'Banana': '89', 'Bean': '31', 'Bread': '238', 'Carrot': '41', 'Cheese': '148', 'Cucumber': '16', 'Egg': '52', 'Grape': '69', 'Kiwi': '61', 'Onion': '40', 'Orange': '47', 'Pasta': '370', 'Pepper': '251', 'Sauce': '100', 'Tomato': '18', 'Watermelon': '30'}

Density: {'Apple': '0.53', 'Banana': '0.63', 'Bean': '1.01', 'Bread': '0.19', 'Carrot': '1.04', 'Cheese': '1.76', 'Cucumber': '0.56', 'Egg': '1.03', 'Grape': '0.39', 'Kiwi': '0.75', 'Onion': '0.48', 'Orange': '0.72', 'Pasta': '0.52', 'Pepper': '0.36', 'Sauce': '1.15', 'Tomato': '1.01', 'Watermelon': '0.64'}


#### Process the images

Rename all the subfolder to only have numbers - From `1-Samsung-S4-Light Environment` to `1`

P.S. The `Qiwi` folder was rename to `Kiwi` manually

In [5]:
classNames = ['Apple', 'Banana', 'Bean', 'Bread', 'Carrot', 'Cheese', 'Cucumber', 'Egg', 'Grape', 'Kiwi', 'Onion', 'Orange', 'Pasta', 'Pepper', 'Sauce', 'Tomato', 'Watermelon']

for className in classNames:
    filePath = f"FooDD/{className}/"
    for folders in listdir(filePath):
        source = os.path.join(filePath, folders)
        if os.path.isdir(source):
            for number in range(1,7):
                if str(number) in folders[0]:
                    dest = os.path.join(filePath, str(number))
                    os.rename(source, dest)
                    print(f"Renamed '{source}' to '{dest}'")
                    break

Renamed 'FooDD/Apple/1-Samsung-S4-Light Environment' to 'FooDD/Apple/1'
Renamed 'FooDD/Apple/2-samsung-S4-Dark Environment' to 'FooDD/Apple/2'
Renamed 'FooDD/Apple/3-IOS-4-Light Environment' to 'FooDD/Apple/3'
Renamed 'FooDD/Apple/4-IOS-4-Dark Environment' to 'FooDD/Apple/4'
Renamed 'FooDD/Apple/5-CanonSD-1400-Light Environment' to 'FooDD/Apple/5'
Renamed 'FooDD/Apple/6-CanonSD-1400-Dark Environment' to 'FooDD/Apple/6'
Renamed 'FooDD/Banana/5-CanonSD-1400-Light Environment' to 'FooDD/Banana/5'
Renamed 'FooDD/Banana/6-CanonSD-1400-Dark Environment' to 'FooDD/Banana/6'
Renamed 'FooDD/Bean/1-Samsung-S4-Light Environment' to 'FooDD/Bean/1'
Renamed 'FooDD/Bean/2-samsung-S4-Dark Environment' to 'FooDD/Bean/2'
Renamed 'FooDD/Bean/3-IOS-4-Light Environment' to 'FooDD/Bean/3'
Renamed 'FooDD/Bean/4-IOS-4-Dark Environment' to 'FooDD/Bean/4'
Renamed 'FooDD/Bread/1-Samsung-S4-Light Environment' to 'FooDD/Bread/1'
Renamed 'FooDD/Bread/2-samsung-S4-Dark Environment' to 'FooDD/Bread/2'
Renamed 'FooDD/

#### 1. Rename all images

Rename all images to `Food_NameDeviceNumber_ImageNumber.jpg` and move to the `Dataset` folder.

E.g. `FooDD/Apple/1/1 (1).jpg` is renamed and moved to `Dataset/Apple1_1.jpg`

In [6]:
classNames = ['Apple', 'Banana', 'Bean', 'Bread', 'Carrot', 'Cheese', 'Cucumber', 'Egg', 'Grape', 'Kiwi', 'Onion', 'Orange', 'Pasta', 'Pepper', 'Sauce', 'Tomato', 'Watermelon']

if not os.path.exists("Dataset"):
    os.mkdir("Dataset")

for className in classNames:
    for number in range(1,7):
        filePath = f"FooDD/{className}/{number}"
        if os.path.exists(filePath):
            for files in listdir(filePath):
                source = os.path.join(filePath, files)
                if os.path.isdir(source):
                    continue # Ignore directories
                    
                if className in files: # It is already named correctly, ignore it
                    continue

                # Correctly name the file
                filename_final = files.replace(" ", "_").replace("(","").replace(")","")

                # Rename and move the file to the Dataset folder
                dest = os.path.normpath("Dataset/"+ className + filename_final)
                os.rename(source, dest)
                print(f"Renamed '{source}' to '{dest}'")

Renamed 'FooDD/Apple/1\1 (1).jpg' to 'Dataset\Apple1_1.jpg'
Renamed 'FooDD/Apple/1\1 (20).jpg' to 'Dataset\Apple1_20.jpg'
Renamed 'FooDD/Apple/1\1 (21).jpg' to 'Dataset\Apple1_21.jpg'
Renamed 'FooDD/Apple/1\1 (22).jpg' to 'Dataset\Apple1_22.jpg'
Renamed 'FooDD/Apple/1\1 (23).jpg' to 'Dataset\Apple1_23.jpg'
Renamed 'FooDD/Apple/1\1 (24).jpg' to 'Dataset\Apple1_24.jpg'
Renamed 'FooDD/Apple/1\1 (25).jpg' to 'Dataset\Apple1_25.jpg'
Renamed 'FooDD/Apple/1\1 (26).jpg' to 'Dataset\Apple1_26.jpg'
Renamed 'FooDD/Apple/1\1 (27).jpg' to 'Dataset\Apple1_27.jpg'
Renamed 'FooDD/Apple/1\1 (28).jpg' to 'Dataset\Apple1_28.jpg'
Renamed 'FooDD/Apple/1\1 (29).jpg' to 'Dataset\Apple1_29.jpg'
Renamed 'FooDD/Apple/1\1 (30).jpg' to 'Dataset\Apple1_30.jpg'
Renamed 'FooDD/Apple/1\1 (31).jpg' to 'Dataset\Apple1_31.jpg'
Renamed 'FooDD/Apple/1\1 (32).jpg' to 'Dataset\Apple1_32.jpg'
Renamed 'FooDD/Apple/1\1 (33).jpg' to 'Dataset\Apple1_33.jpg'
Renamed 'FooDD/Apple/1\1 (34).jpg' to 'Dataset\Apple1_34.jpg'
Renamed 'F

Renamed 'FooDD/Apple/3\3 (70).jpg' to 'Dataset\Apple3_70.jpg'
Renamed 'FooDD/Apple/3\3 (71).jpg' to 'Dataset\Apple3_71.jpg'
Renamed 'FooDD/Apple/3\3 (72).jpg' to 'Dataset\Apple3_72.jpg'
Renamed 'FooDD/Apple/3\3 (73).jpg' to 'Dataset\Apple3_73.jpg'
Renamed 'FooDD/Apple/3\3 (74).jpg' to 'Dataset\Apple3_74.jpg'
Renamed 'FooDD/Apple/3\3 (75).jpg' to 'Dataset\Apple3_75.jpg'
Renamed 'FooDD/Apple/3\3 (76).jpg' to 'Dataset\Apple3_76.jpg'
Renamed 'FooDD/Apple/3\3 (77).jpg' to 'Dataset\Apple3_77.jpg'
Renamed 'FooDD/Apple/3\3 (78).jpg' to 'Dataset\Apple3_78.jpg'
Renamed 'FooDD/Apple/3\3 (79).jpg' to 'Dataset\Apple3_79.jpg'
Renamed 'FooDD/Apple/3\3 (8).jpg' to 'Dataset\Apple3_8.jpg'
Renamed 'FooDD/Apple/3\3 (80).jpg' to 'Dataset\Apple3_80.jpg'
Renamed 'FooDD/Apple/3\3 (81).jpg' to 'Dataset\Apple3_81.jpg'
Renamed 'FooDD/Apple/3\3 (82).jpg' to 'Dataset\Apple3_82.jpg'
Renamed 'FooDD/Apple/3\3 (83).jpg' to 'Dataset\Apple3_83.jpg'
Renamed 'FooDD/Apple/3\3 (84).jpg' to 'Dataset\Apple3_84.jpg'
Renamed 'F

Renamed 'FooDD/Banana/5\5 (16).JPG' to 'Dataset\Banana5_16.JPG'
Renamed 'FooDD/Banana/5\5 (17).JPG' to 'Dataset\Banana5_17.JPG'
Renamed 'FooDD/Banana/5\5 (18).JPG' to 'Dataset\Banana5_18.JPG'
Renamed 'FooDD/Banana/5\5 (19).JPG' to 'Dataset\Banana5_19.JPG'
Renamed 'FooDD/Banana/5\5 (2).JPG' to 'Dataset\Banana5_2.JPG'
Renamed 'FooDD/Banana/5\5 (20).JPG' to 'Dataset\Banana5_20.JPG'
Renamed 'FooDD/Banana/5\5 (21).JPG' to 'Dataset\Banana5_21.JPG'
Renamed 'FooDD/Banana/5\5 (22).JPG' to 'Dataset\Banana5_22.JPG'
Renamed 'FooDD/Banana/5\5 (23).JPG' to 'Dataset\Banana5_23.JPG'
Renamed 'FooDD/Banana/5\5 (24).JPG' to 'Dataset\Banana5_24.JPG'
Renamed 'FooDD/Banana/5\5 (25).JPG' to 'Dataset\Banana5_25.JPG'
Renamed 'FooDD/Banana/5\5 (26).JPG' to 'Dataset\Banana5_26.JPG'
Renamed 'FooDD/Banana/5\5 (27).JPG' to 'Dataset\Banana5_27.JPG'
Renamed 'FooDD/Banana/5\5 (28).JPG' to 'Dataset\Banana5_28.JPG'
Renamed 'FooDD/Banana/5\5 (29).JPG' to 'Dataset\Banana5_29.JPG'
Renamed 'FooDD/Banana/5\5 (3).JPG' to 'Dat

Renamed 'FooDD/Bean/2\2 (13).jpg' to 'Dataset\Bean2_13.jpg'
Renamed 'FooDD/Bean/2\2 (14).jpg' to 'Dataset\Bean2_14.jpg'
Renamed 'FooDD/Bean/2\2 (15).jpg' to 'Dataset\Bean2_15.jpg'
Renamed 'FooDD/Bean/2\2 (16).jpg' to 'Dataset\Bean2_16.jpg'
Renamed 'FooDD/Bean/2\2 (17).jpg' to 'Dataset\Bean2_17.jpg'
Renamed 'FooDD/Bean/2\2 (18).jpg' to 'Dataset\Bean2_18.jpg'
Renamed 'FooDD/Bean/2\2 (19).jpg' to 'Dataset\Bean2_19.jpg'
Renamed 'FooDD/Bean/2\2 (2).jpg' to 'Dataset\Bean2_2.jpg'
Renamed 'FooDD/Bean/2\2 (20).jpg' to 'Dataset\Bean2_20.jpg'
Renamed 'FooDD/Bean/2\2 (21).jpg' to 'Dataset\Bean2_21.jpg'
Renamed 'FooDD/Bean/2\2 (22).jpg' to 'Dataset\Bean2_22.jpg'
Renamed 'FooDD/Bean/2\2 (23).jpg' to 'Dataset\Bean2_23.jpg'
Renamed 'FooDD/Bean/2\2 (24).jpg' to 'Dataset\Bean2_24.jpg'
Renamed 'FooDD/Bean/2\2 (25).jpg' to 'Dataset\Bean2_25.jpg'
Renamed 'FooDD/Bean/2\2 (26).jpg' to 'Dataset\Bean2_26.jpg'
Renamed 'FooDD/Bean/2\2 (27).jpg' to 'Dataset\Bean2_27.jpg'
Renamed 'FooDD/Bean/2\2 (28).jpg' to 'Data

Renamed 'FooDD/Bean/4\4 (37).jpg' to 'Dataset\Bean4_37.jpg'
Renamed 'FooDD/Bean/4\4 (38).jpg' to 'Dataset\Bean4_38.jpg'
Renamed 'FooDD/Bean/4\4 (39).jpg' to 'Dataset\Bean4_39.jpg'
Renamed 'FooDD/Bean/4\4 (4).jpg' to 'Dataset\Bean4_4.jpg'
Renamed 'FooDD/Bean/4\4 (40).jpg' to 'Dataset\Bean4_40.jpg'
Renamed 'FooDD/Bean/4\4 (41).jpg' to 'Dataset\Bean4_41.jpg'
Renamed 'FooDD/Bean/4\4 (42).jpg' to 'Dataset\Bean4_42.jpg'
Renamed 'FooDD/Bean/4\4 (43).jpg' to 'Dataset\Bean4_43.jpg'
Renamed 'FooDD/Bean/4\4 (44).jpg' to 'Dataset\Bean4_44.jpg'
Renamed 'FooDD/Bean/4\4 (45).jpg' to 'Dataset\Bean4_45.jpg'
Renamed 'FooDD/Bean/4\4 (46).jpg' to 'Dataset\Bean4_46.jpg'
Renamed 'FooDD/Bean/4\4 (47).jpg' to 'Dataset\Bean4_47.jpg'
Renamed 'FooDD/Bean/4\4 (48).jpg' to 'Dataset\Bean4_48.jpg'
Renamed 'FooDD/Bean/4\4 (49).jpg' to 'Dataset\Bean4_49.jpg'
Renamed 'FooDD/Bean/4\4 (5).jpg' to 'Dataset\Bean4_5.jpg'
Renamed 'FooDD/Bean/4\4 (50).jpg' to 'Dataset\Bean4_50.jpg'
Renamed 'FooDD/Bean/4\4 (51).jpg' to 'Datase

...

Renamed 'FooDD/Pepper/5\5 (4).JPG' to 'Dataset\Pepper5_4.JPG'
Renamed 'FooDD/Pepper/5\5 (40).JPG' to 'Dataset\Pepper5_40.JPG'
Renamed 'FooDD/Pepper/5\5 (41).JPG' to 'Dataset\Pepper5_41.JPG'
Renamed 'FooDD/Pepper/5\5 (42).JPG' to 'Dataset\Pepper5_42.JPG'
Renamed 'FooDD/Pepper/5\5 (43).JPG' to 'Dataset\Pepper5_43.JPG'
Renamed 'FooDD/Pepper/5\5 (44).JPG' to 'Dataset\Pepper5_44.JPG'
Renamed 'FooDD/Pepper/5\5 (45).JPG' to 'Dataset\Pepper5_45.JPG'
Renamed 'FooDD/Pepper/5\5 (46).JPG' to 'Dataset\Pepper5_46.JPG'
Renamed 'FooDD/Pepper/5\5 (47).JPG' to 'Dataset\Pepper5_47.JPG'
Renamed 'FooDD/Pepper/5\5 (48).JPG' to 'Dataset\Pepper5_48.JPG'
Renamed 'FooDD/Pepper/5\5 (49).JPG' to 'Dataset\Pepper5_49.JPG'
Renamed 'FooDD/Pepper/5\5 (5).JPG' to 'Dataset\Pepper5_5.JPG'
Renamed 'FooDD/Pepper/5\5 (50).JPG' to 'Dataset\Pepper5_50.JPG'
Renamed 'FooDD/Pepper/5\5 (51).JPG' to 'Dataset\Pepper5_51.JPG'
Renamed 'FooDD/Pepper/5\5 (52).JPG' to 'Dataset\Pepper5_52.JPG'
Renamed 'FooDD/Pepper/5\5 (53).JPG' to 'Data

Renamed 'FooDD/Sauce/2\2 (4).jpg' to 'Dataset\Sauce2_4.jpg'
Renamed 'FooDD/Sauce/2\2 (5).jpg' to 'Dataset\Sauce2_5.jpg'
Renamed 'FooDD/Sauce/2\2 (6).jpg' to 'Dataset\Sauce2_6.jpg'
Renamed 'FooDD/Sauce/2\2 (7).jpg' to 'Dataset\Sauce2_7.jpg'
Renamed 'FooDD/Sauce/2\2 (8).jpg' to 'Dataset\Sauce2_8.jpg'
Renamed 'FooDD/Sauce/2\2 (9).jpg' to 'Dataset\Sauce2_9.jpg'
Renamed 'FooDD/Sauce/3\3 (1).jpg' to 'Dataset\Sauce3_1.jpg'
Renamed 'FooDD/Sauce/3\3 (10).jpg' to 'Dataset\Sauce3_10.jpg'
Renamed 'FooDD/Sauce/3\3 (11).jpg' to 'Dataset\Sauce3_11.jpg'
Renamed 'FooDD/Sauce/3\3 (12).jpg' to 'Dataset\Sauce3_12.jpg'
Renamed 'FooDD/Sauce/3\3 (13).jpg' to 'Dataset\Sauce3_13.jpg'
Renamed 'FooDD/Sauce/3\3 (14).jpg' to 'Dataset\Sauce3_14.jpg'
Renamed 'FooDD/Sauce/3\3 (15).jpg' to 'Dataset\Sauce3_15.jpg'
Renamed 'FooDD/Sauce/3\3 (16).jpg' to 'Dataset\Sauce3_16.jpg'
Renamed 'FooDD/Sauce/3\3 (17).jpg' to 'Dataset\Sauce3_17.jpg'
Renamed 'FooDD/Sauce/3\3 (18).jpg' to 'Dataset\Sauce3_18.jpg'
Renamed 'FooDD/Sauce/3

Renamed 'FooDD/Tomato/1\1 (54).jpg' to 'Dataset\Tomato1_54.jpg'
Renamed 'FooDD/Tomato/1\1 (55).jpg' to 'Dataset\Tomato1_55.jpg'
Renamed 'FooDD/Tomato/1\1 (56).jpg' to 'Dataset\Tomato1_56.jpg'
Renamed 'FooDD/Tomato/1\1 (57).jpg' to 'Dataset\Tomato1_57.jpg'
Renamed 'FooDD/Tomato/1\1 (58).jpg' to 'Dataset\Tomato1_58.jpg'
Renamed 'FooDD/Tomato/1\1 (6).jpg' to 'Dataset\Tomato1_6.jpg'
Renamed 'FooDD/Tomato/1\1 (7).jpg' to 'Dataset\Tomato1_7.jpg'
Renamed 'FooDD/Tomato/1\1 (8).jpg' to 'Dataset\Tomato1_8.jpg'
Renamed 'FooDD/Tomato/1\1 (9).jpg' to 'Dataset\Tomato1_9.jpg'
Renamed 'FooDD/Tomato/2\2 (1).jpg' to 'Dataset\Tomato2_1.jpg'
Renamed 'FooDD/Tomato/2\2 (10).jpg' to 'Dataset\Tomato2_10.jpg'
Renamed 'FooDD/Tomato/2\2 (11).jpg' to 'Dataset\Tomato2_11.jpg'
Renamed 'FooDD/Tomato/2\2 (12).jpg' to 'Dataset\Tomato2_12.jpg'
Renamed 'FooDD/Tomato/2\2 (13).jpg' to 'Dataset\Tomato2_13.jpg'
Renamed 'FooDD/Tomato/2\2 (14).jpg' to 'Dataset\Tomato2_14.jpg'
Renamed 'FooDD/Tomato/2\2 (15).jpg' to 'Dataset\To

Renamed 'FooDD/Tomato/3\3 (91).jpg' to 'Dataset\Tomato3_91.jpg'
Renamed 'FooDD/Tomato/3\3 (92).jpg' to 'Dataset\Tomato3_92.jpg'
Renamed 'FooDD/Tomato/3\3 (93).jpg' to 'Dataset\Tomato3_93.jpg'
Renamed 'FooDD/Tomato/3\3 (94).jpg' to 'Dataset\Tomato3_94.jpg'
Renamed 'FooDD/Tomato/3\3 (95).jpg' to 'Dataset\Tomato3_95.jpg'
Renamed 'FooDD/Tomato/3\3 (96).jpg' to 'Dataset\Tomato3_96.jpg'
Renamed 'FooDD/Tomato/4\4 (1).jpg' to 'Dataset\Tomato4_1.jpg'
Renamed 'FooDD/Tomato/4\4 (10).jpg' to 'Dataset\Tomato4_10.jpg'
Renamed 'FooDD/Tomato/4\4 (11).jpg' to 'Dataset\Tomato4_11.jpg'
Renamed 'FooDD/Tomato/4\4 (12).jpg' to 'Dataset\Tomato4_12.jpg'
Renamed 'FooDD/Tomato/4\4 (13).jpg' to 'Dataset\Tomato4_13.jpg'
Renamed 'FooDD/Tomato/4\4 (14).jpg' to 'Dataset\Tomato4_14.jpg'
Renamed 'FooDD/Tomato/4\4 (15).jpg' to 'Dataset\Tomato4_15.jpg'
Renamed 'FooDD/Tomato/4\4 (16).jpg' to 'Dataset\Tomato4_16.jpg'
Renamed 'FooDD/Tomato/4\4 (17).jpg' to 'Dataset\Tomato4_17.jpg'
Renamed 'FooDD/Tomato/4\4 (18).jpg' to 'Da

Renamed 'FooDD/Watermelon/6\6 (19).JPG' to 'Dataset\Watermelon6_19.JPG'
Renamed 'FooDD/Watermelon/6\6 (2).JPG' to 'Dataset\Watermelon6_2.JPG'
Renamed 'FooDD/Watermelon/6\6 (20).JPG' to 'Dataset\Watermelon6_20.JPG'
Renamed 'FooDD/Watermelon/6\6 (21).JPG' to 'Dataset\Watermelon6_21.JPG'
Renamed 'FooDD/Watermelon/6\6 (22).JPG' to 'Dataset\Watermelon6_22.JPG'
Renamed 'FooDD/Watermelon/6\6 (23).JPG' to 'Dataset\Watermelon6_23.JPG'
Renamed 'FooDD/Watermelon/6\6 (24).JPG' to 'Dataset\Watermelon6_24.JPG'
Renamed 'FooDD/Watermelon/6\6 (25).JPG' to 'Dataset\Watermelon6_25.JPG'
Renamed 'FooDD/Watermelon/6\6 (26).JPG' to 'Dataset\Watermelon6_26.JPG'
Renamed 'FooDD/Watermelon/6\6 (27).JPG' to 'Dataset\Watermelon6_27.JPG'
Renamed 'FooDD/Watermelon/6\6 (28).JPG' to 'Dataset\Watermelon6_28.JPG'
Renamed 'FooDD/Watermelon/6\6 (29).JPG' to 'Dataset\Watermelon6_29.JPG'
Renamed 'FooDD/Watermelon/6\6 (3).JPG' to 'Dataset\Watermelon6_3.JPG'
Renamed 'FooDD/Watermelon/6\6 (30).JPG' to 'Dataset\Watermelon6_30.J

`updateAllImages` - Get all the images in the Dataset folder.


##### 2. Scale all images down to 1000px on the smallest side

`image_resize` - Resizes a single image to a specified size. By default it is 1000px on the smallest side

`resizeAllImages` - Iterates over the full list of images and resizes each image. It stores the final image in a new directory called `Resized`. 

##### 3. Crop all images to the same size (1000x1000px)

`cropImage` - Crops a single image to a specified size. By default it is 1000px

`cropAllImages` - Iterates over the full list of images and crops each image. It stores the final image in a new directory called `Cropped`. 


By resizing first and then cropping, we ensure that we lose the minimal amount of information in the image, compared to cropping first and then resizing.

In [7]:
allImages = []

def updateAllImages(folderPath="Dataset"):
    images = listdir(folderPath)

    global allImages
    allImages = []
    
    for image in images:
        image_path = os.path.join(folderPath, image)
        if os.path.isfile(image_path):
            allImages.append(image_path)
        
def image_resize(image, size=1000, inter = cv2.INTER_AREA):
    dim = None
    (height, width) = image.shape[:2]
    
    # Resize the image based on the smallest size
    # This allows the image to be cropped later
    smallestDimension = "height"
    if width < height:
        smallestDimension = "width"

    if smallestDimension == "height":
        # calculate the ratio of the height and construct the dimensions
        r = size / float(height)
        dim = (int(width * r), size)

    # otherwise, the smallest dimension is width
    else:
        # calculate the ratio of the width and construct the dimensions
        r = size / float(width)
        dim = (size, int(height * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)
    
    # return the resized image
    return resized
        


def resizeAllImages(dirName = "Resized"):
    for image in allImages:
        dirPath = os.path.normpath(os.path.join(image, "..", dirName))
        if not os.path.exists(dirPath): # Create dir if needed
            os.mkdir(dirPath)
        imagedata = cv2.imread(image)
        try:
            imagedata.shape[:2]
        except:
            continue
        
        print(f"Resizing '{image}'")        
        image_resized = image_resize(imagedata)
        # Uncomment the next line to do crop after resize
        # image_resized = cropImage(image_resized)
        
        imagePath = os.path.join(dirPath, os.path.basename(image))
        
        cv2.imwrite(imagePath, image_resized)
        
        
def cropImage(image, size=1000):
    startx = image.shape[1]//2-(size//2)
    starty = image.shape[0]//2-(size//2)   

    return image[starty:starty+size, startx:startx+size]
        

def cropAllImages(dirName = "Cropped"):
    for image in allImages:
        dirPath = os.path.normpath(os.path.join(image, "..", dirName))
        if not os.path.exists(dirPath):
            os.mkdir(dirPath)
            
        imagedata = cv2.imread(image)
        try:
            imagedata.shape[:2]
        except:
            continue
        
        print(f"Cropping '{image}'")        
        image_cropped = cropImage(imagedata)
        
        imagePath = os.path.join(dirPath, os.path.basename(image))
        
        cv2.imwrite(imagePath, image_cropped)
       

Run the above code and ensure that everything works properly

The code can be optimized by doing both actions (resize and crop) at the same time in a single function, instead of doing one and then the other.

In [8]:
updateAllImages()
resizeAllImages("") # Save in the same folder
cropAllImages("") # Save in the same folder

Resizing 'Dataset\Apple1_1.jpg'
Resizing 'Dataset\Apple1_20.jpg'
Resizing 'Dataset\Apple1_21.jpg'
Resizing 'Dataset\Apple1_22.jpg'
Resizing 'Dataset\Apple1_23.jpg'
Resizing 'Dataset\Apple1_24.jpg'
Resizing 'Dataset\Apple1_25.jpg'
Resizing 'Dataset\Apple1_26.jpg'
Resizing 'Dataset\Apple1_27.jpg'
Resizing 'Dataset\Apple1_28.jpg'
Resizing 'Dataset\Apple1_29.jpg'
Resizing 'Dataset\Apple1_30.jpg'
Resizing 'Dataset\Apple1_31.jpg'
Resizing 'Dataset\Apple1_32.jpg'
Resizing 'Dataset\Apple1_33.jpg'
Resizing 'Dataset\Apple1_34.jpg'
Resizing 'Dataset\Apple1_35.jpg'
Resizing 'Dataset\Apple1_36.jpg'
Resizing 'Dataset\Apple1_37.jpg'
Resizing 'Dataset\Apple1_38.jpg'
Resizing 'Dataset\Apple1_39.jpg'
Resizing 'Dataset\Apple1_40.jpg'
Resizing 'Dataset\Apple1_41.jpg'
Resizing 'Dataset\Apple1_42.jpg'
Resizing 'Dataset\Apple1_43.jpg'
Resizing 'Dataset\Apple1_44.jpg'
Resizing 'Dataset\Apple1_45.jpg'
Resizing 'Dataset\Apple1_46.jpg'
Resizing 'Dataset\Apple1_47.jpg'
Resizing 'Dataset\Apple1_48.jpg'
Resizing 'D

Resizing 'Dataset\Apple3_85.jpg'
Resizing 'Dataset\Apple3_86.jpg'
Resizing 'Dataset\Apple3_87.jpg'
Resizing 'Dataset\Apple3_88.jpg'
Resizing 'Dataset\Apple3_89.jpg'
Resizing 'Dataset\Apple3_9.jpg'
Resizing 'Dataset\Apple3_90.jpg'
Resizing 'Dataset\Apple3_91.jpg'
Resizing 'Dataset\Apple3_92.jpg'
Resizing 'Dataset\Apple3_93.jpg'
Resizing 'Dataset\Apple3_94.jpg'
Resizing 'Dataset\Apple3_95.jpg'
Resizing 'Dataset\Apple3_96.jpg'
Resizing 'Dataset\Apple3_97.jpg'
Resizing 'Dataset\Apple3_98.jpg'
Resizing 'Dataset\Apple3_99.jpg'
Resizing 'Dataset\Apple4_1.jpg'
Resizing 'Dataset\Apple4_10.jpg'
Resizing 'Dataset\Apple4_11.jpg'
Resizing 'Dataset\Apple4_12.jpg'
Resizing 'Dataset\Apple4_13.jpg'
Resizing 'Dataset\Apple4_14.jpg'
Resizing 'Dataset\Apple4_15.jpg'
Resizing 'Dataset\Apple4_16.jpg'
Resizing 'Dataset\Apple4_17.jpg'
Resizing 'Dataset\Apple4_18.jpg'
Resizing 'Dataset\Apple4_19.jpg'
Resizing 'Dataset\Apple4_2.jpg'
Resizing 'Dataset\Apple4_20.jpg'
Resizing 'Dataset\Apple4_21.jpg'
Resizing 'Dat

Resizing 'Dataset\Banana5_65.JPG'
Resizing 'Dataset\Banana5_66.JPG'
Resizing 'Dataset\Banana5_67.JPG'
Resizing 'Dataset\Banana5_68.JPG'
Resizing 'Dataset\Banana5_69.JPG'
Resizing 'Dataset\Banana5_7.JPG'
Resizing 'Dataset\Banana5_70.JPG'
Resizing 'Dataset\Banana5_71.JPG'
Resizing 'Dataset\Banana5_72.JPG'
Resizing 'Dataset\Banana5_73.JPG'
Resizing 'Dataset\Banana5_74.JPG'
Resizing 'Dataset\Banana5_75.JPG'
Resizing 'Dataset\Banana5_76.JPG'
Resizing 'Dataset\Banana5_77.JPG'
Resizing 'Dataset\Banana5_78.JPG'
Resizing 'Dataset\Banana5_79.JPG'
Resizing 'Dataset\Banana5_8.JPG'
Resizing 'Dataset\Banana5_80.JPG'
Resizing 'Dataset\Banana5_81.JPG'
Resizing 'Dataset\Banana5_82.JPG'
Resizing 'Dataset\Banana5_83.JPG'
Resizing 'Dataset\Banana5_9.JPG'
Resizing 'Dataset\Banana6_1.JPG'
Resizing 'Dataset\Banana6_10.JPG'
Resizing 'Dataset\Banana6_11.JPG'
Resizing 'Dataset\Banana6_12.JPG'
Resizing 'Dataset\Banana6_13.JPG'
Resizing 'Dataset\Banana6_14.JPG'
Resizing 'Dataset\Banana6_15.JPG'
Resizing 'Dataset\

Resizing 'Dataset\Bean3_59.jpg'
Resizing 'Dataset\Bean3_6.jpg'
Resizing 'Dataset\Bean3_60.jpg'
Resizing 'Dataset\Bean3_61.jpg'
Resizing 'Dataset\Bean3_62.jpg'
Resizing 'Dataset\Bean3_63.jpg'
Resizing 'Dataset\Bean3_64.jpg'
Resizing 'Dataset\Bean3_65.jpg'
Resizing 'Dataset\Bean3_66.jpg'
Resizing 'Dataset\Bean3_67.jpg'
Resizing 'Dataset\Bean3_68.jpg'
Resizing 'Dataset\Bean3_69.jpg'
Resizing 'Dataset\Bean3_7.jpg'
Resizing 'Dataset\Bean3_70.jpg'
Resizing 'Dataset\Bean3_71.jpg'
Resizing 'Dataset\Bean3_72.jpg'
Resizing 'Dataset\Bean3_73.jpg'
Resizing 'Dataset\Bean3_74.jpg'
Resizing 'Dataset\Bean3_75.jpg'
Resizing 'Dataset\Bean3_76.jpg'
Resizing 'Dataset\Bean3_77.jpg'
Resizing 'Dataset\Bean3_78.jpg'
Resizing 'Dataset\Bean3_79.jpg'
Resizing 'Dataset\Bean3_8.jpg'
Resizing 'Dataset\Bean3_80.jpg'
Resizing 'Dataset\Bean3_81.jpg'
Resizing 'Dataset\Bean3_82.jpg'
Resizing 'Dataset\Bean3_83.jpg'
Resizing 'Dataset\Bean3_84.jpg'
Resizing 'Dataset\Bean3_85.jpg'
Resizing 'Dataset\Bean3_9.jpg'
Resizing 'Da

Resizing 'Dataset\Cheese1_6.jpg'
Resizing 'Dataset\Cheese1_7.jpg'
Resizing 'Dataset\Cheese1_8.jpg'
Resizing 'Dataset\Cheese1_9.jpg'
Resizing 'Dataset\Cheese2_1.jpg'
Resizing 'Dataset\Cheese2_10.jpg'
Resizing 'Dataset\Cheese2_11.jpg'
Resizing 'Dataset\Cheese2_12.jpg'
Resizing 'Dataset\Cheese2_13.jpg'
Resizing 'Dataset\Cheese2_14.jpg'
Resizing 'Dataset\Cheese2_15.jpg'
Resizing 'Dataset\Cheese2_16.jpg'
Resizing 'Dataset\Cheese2_17.jpg'
Resizing 'Dataset\Cheese2_18.jpg'
Resizing 'Dataset\Cheese2_19.jpg'
Resizing 'Dataset\Cheese2_2.jpg'
Resizing 'Dataset\Cheese2_20.jpg'
Resizing 'Dataset\Cheese2_21.jpg'
Resizing 'Dataset\Cheese2_22.jpg'
Resizing 'Dataset\Cheese2_23.jpg'
Resizing 'Dataset\Cheese2_24.jpg'
Resizing 'Dataset\Cheese2_25.jpg'
Resizing 'Dataset\Cheese2_26.jpg'
Resizing 'Dataset\Cheese2_27.jpg'
Resizing 'Dataset\Cheese2_28.jpg'
Resizing 'Dataset\Cheese2_29.jpg'
Resizing 'Dataset\Cheese2_3.jpg'
Resizing 'Dataset\Cheese2_30.jpg'
Resizing 'Dataset\Cheese2_31.jpg'
Resizing 'Dataset\Che

...

Cropping 'Dataset\Onion4_4.jpg'
Cropping 'Dataset\Onion4_40.jpg'
Cropping 'Dataset\Onion4_41.jpg'
Cropping 'Dataset\Onion4_42.jpg'
Cropping 'Dataset\Onion4_43.jpg'
Cropping 'Dataset\Onion4_44.jpg'
Cropping 'Dataset\Onion4_45.jpg'
Cropping 'Dataset\Onion4_46.jpg'
Cropping 'Dataset\Onion4_47.jpg'
Cropping 'Dataset\Onion4_48.jpg'
Cropping 'Dataset\Onion4_49.jpg'
Cropping 'Dataset\Onion4_5.jpg'
Cropping 'Dataset\Onion4_50.jpg'
Cropping 'Dataset\Onion4_51.jpg'
Cropping 'Dataset\Onion4_52.jpg'
Cropping 'Dataset\Onion4_53.jpg'
Cropping 'Dataset\Onion4_54.jpg'
Cropping 'Dataset\Onion4_55.jpg'
Cropping 'Dataset\Onion4_56.jpg'
Cropping 'Dataset\Onion4_57.jpg'
Cropping 'Dataset\Onion4_58.jpg'
Cropping 'Dataset\Onion4_59.jpg'
Cropping 'Dataset\Onion4_6.jpg'
Cropping 'Dataset\Onion4_60.jpg'
Cropping 'Dataset\Onion4_61.jpg'
Cropping 'Dataset\Onion4_62.jpg'
Cropping 'Dataset\Onion4_63.jpg'
Cropping 'Dataset\Onion4_64.jpg'
Cropping 'Dataset\Onion4_65.jpg'
Cropping 'Dataset\Onion4_66.jpg'
Cropping 'Dat

Cropping 'Dataset\Orange4_18.jpg'
Cropping 'Dataset\Orange4_19.jpg'
Cropping 'Dataset\Orange4_2.jpg'
Cropping 'Dataset\Orange4_20.jpg'
Cropping 'Dataset\Orange4_21.jpg'
Cropping 'Dataset\Orange4_22.jpg'
Cropping 'Dataset\Orange4_23.jpg'
Cropping 'Dataset\Orange4_24.jpg'
Cropping 'Dataset\Orange4_25.jpg'
Cropping 'Dataset\Orange4_26.jpg'
Cropping 'Dataset\Orange4_27.jpg'
Cropping 'Dataset\Orange4_28.jpg'
Cropping 'Dataset\Orange4_29.jpg'
Cropping 'Dataset\Orange4_3.jpg'
Cropping 'Dataset\Orange4_30.jpg'
Cropping 'Dataset\Orange4_31.jpg'
Cropping 'Dataset\Orange4_32.jpg'
Cropping 'Dataset\Orange4_33.jpg'
Cropping 'Dataset\Orange4_34.jpg'
Cropping 'Dataset\Orange4_35.jpg'
Cropping 'Dataset\Orange4_36.jpg'
Cropping 'Dataset\Orange4_37.jpg'
Cropping 'Dataset\Orange4_38.jpg'
Cropping 'Dataset\Orange4_39.jpg'
Cropping 'Dataset\Orange4_4.jpg'
Cropping 'Dataset\Orange4_40.jpg'
Cropping 'Dataset\Orange4_41.jpg'
Cropping 'Dataset\Orange4_42.jpg'
Cropping 'Dataset\Orange4_43.jpg'
Cropping 'Dataset

Cropping 'Dataset\Pepper5_14.JPG'
Cropping 'Dataset\Pepper5_15.JPG'
Cropping 'Dataset\Pepper5_16.JPG'
Cropping 'Dataset\Pepper5_17.JPG'
Cropping 'Dataset\Pepper5_18.JPG'
Cropping 'Dataset\Pepper5_19.JPG'
Cropping 'Dataset\Pepper5_2.JPG'
Cropping 'Dataset\Pepper5_20.JPG'
Cropping 'Dataset\Pepper5_21.JPG'
Cropping 'Dataset\Pepper5_22.JPG'
Cropping 'Dataset\Pepper5_23.JPG'
Cropping 'Dataset\Pepper5_24.JPG'
Cropping 'Dataset\Pepper5_25.JPG'
Cropping 'Dataset\Pepper5_26.JPG'
Cropping 'Dataset\Pepper5_27.JPG'
Cropping 'Dataset\Pepper5_28.JPG'
Cropping 'Dataset\Pepper5_29.JPG'
Cropping 'Dataset\Pepper5_3.JPG'
Cropping 'Dataset\Pepper5_30.JPG'
Cropping 'Dataset\Pepper5_31.JPG'
Cropping 'Dataset\Pepper5_32.JPG'
Cropping 'Dataset\Pepper5_33.JPG'
Cropping 'Dataset\Pepper5_34.JPG'
Cropping 'Dataset\Pepper5_35.JPG'
Cropping 'Dataset\Pepper5_36.JPG'
Cropping 'Dataset\Pepper5_37.JPG'
Cropping 'Dataset\Pepper5_38.JPG'
Cropping 'Dataset\Pepper5_39.JPG'
Cropping 'Dataset\Pepper5_4.JPG'
Cropping 'Dataset

Cropping 'Dataset\Sauce4_12.jpg'
Cropping 'Dataset\Sauce4_13.jpg'
Cropping 'Dataset\Sauce4_14.jpg'
Cropping 'Dataset\Sauce4_15.jpg'
Cropping 'Dataset\Sauce4_16.jpg'
Cropping 'Dataset\Sauce4_17.jpg'
Cropping 'Dataset\Sauce4_18.jpg'
Cropping 'Dataset\Sauce4_19.jpg'
Cropping 'Dataset\Sauce4_2.jpg'
Cropping 'Dataset\Sauce4_20.jpg'
Cropping 'Dataset\Sauce4_21.jpg'
Cropping 'Dataset\Sauce4_22.jpg'
Cropping 'Dataset\Sauce4_23.jpg'
Cropping 'Dataset\Sauce4_24.jpg'
Cropping 'Dataset\Sauce4_25.jpg'
Cropping 'Dataset\Sauce4_26.jpg'
Cropping 'Dataset\Sauce4_27.jpg'
Cropping 'Dataset\Sauce4_28.jpg'
Cropping 'Dataset\Sauce4_29.jpg'
Cropping 'Dataset\Sauce4_3.jpg'
Cropping 'Dataset\Sauce4_30.jpg'
Cropping 'Dataset\Sauce4_31.jpg'
Cropping 'Dataset\Sauce4_32.jpg'
Cropping 'Dataset\Sauce4_33.jpg'
Cropping 'Dataset\Sauce4_34.jpg'
Cropping 'Dataset\Sauce4_35.jpg'
Cropping 'Dataset\Sauce4_36.jpg'
Cropping 'Dataset\Sauce4_37.jpg'
Cropping 'Dataset\Sauce4_38.jpg'
Cropping 'Dataset\Sauce4_39.jpg'
Cropping 'Da

Cropping 'Dataset\Tomato3_96.jpg'
Cropping 'Dataset\Tomato4_1.jpg'
Cropping 'Dataset\Tomato4_10.jpg'
Cropping 'Dataset\Tomato4_11.jpg'
Cropping 'Dataset\Tomato4_12.jpg'
Cropping 'Dataset\Tomato4_13.jpg'
Cropping 'Dataset\Tomato4_14.jpg'
Cropping 'Dataset\Tomato4_15.jpg'
Cropping 'Dataset\Tomato4_16.jpg'
Cropping 'Dataset\Tomato4_17.jpg'
Cropping 'Dataset\Tomato4_18.jpg'
Cropping 'Dataset\Tomato4_19.jpg'
Cropping 'Dataset\Tomato4_2.jpg'
Cropping 'Dataset\Tomato4_20.jpg'
Cropping 'Dataset\Tomato4_21.jpg'
Cropping 'Dataset\Tomato4_22.jpg'
Cropping 'Dataset\Tomato4_23.jpg'
Cropping 'Dataset\Tomato4_24.jpg'
Cropping 'Dataset\Tomato4_25.jpg'
Cropping 'Dataset\Tomato4_26.jpg'
Cropping 'Dataset\Tomato4_27.jpg'
Cropping 'Dataset\Tomato4_28.jpg'
Cropping 'Dataset\Tomato4_29.jpg'
Cropping 'Dataset\Tomato4_3.jpg'
Cropping 'Dataset\Tomato4_30.jpg'
Cropping 'Dataset\Tomato4_4.jpg'
Cropping 'Dataset\Tomato4_5.jpg'
Cropping 'Dataset\Tomato4_6.jpg'
Cropping 'Dataset\Tomato4_7.jpg'
Cropping 'Dataset\Tom

After manually splitting the dataset into training and testing folders (80%:20%), we can run the code below to create the labels file.

The reason why the split is done manually, is to have a look at the data and check for any anomalies before training the model.

In [9]:
def createCsvFile(filename, directory):
    if not filename.endswith(".csv"):
        filename = filename + ".csv"
    fileLocation = os.path.join(directory, ".." ,filename)
    with open(fileLocation, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["File","Label"])
        for items in os.walk(directory):
            fileNames = items[2]
            for file in fileNames:
                className = [k for k in classNames if k in file]
                label = [k for k,v in class_label.items() if v in className]
                writer.writerow([file,label])
                
# createCsvFile("train_labels.csv","Dataset/ImageClassification/train")
# createCsvFile("test_labels.csv","Dataset/ImageClassification/test")
#
# Open the csv and remove the [] from the values
#